# Analyse et Modélisation des Prix de Véhicules
Créer un modèle pour prédire le prix d’un véhicule à partir de caractéristiques techniques et visuelles, avec un dataset sale, non structuré et plein de chaînes de caractères mal formatées.

🔗 Dataset :
👉 https://www.kaggle.com/datasets/CooperUnion/cardataset

## 1. Compréhension et audit du dataset
- Charger les données
- Identifier les types de variables (numériques, catégorielles, chaînes)
- Lister les incohérences et valeurs manquantes
- Questions : colonnes utiles, corrélations potentielles, doublons/rédundances



In [5]:
pip install pandas

  Using cached pandas-2.2.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp313-cp313-win_amd64.whl (11.5 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy

   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ----------------------- ---------------- 7.3/12.6 MB 35.5 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 31.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np
# Charger le dataset
# df = pd.read_csv('path_to_data.csv')

In [11]:
# Charger le dataset
file_path = r"C:\Users\kapna\Desktop\DIGITAL SCHOOL OF PARIS\PYTHON\mini_projet\data.csv"
df = pd.read_csv(file_path, encoding='utf-8', low_memory=False)
print(df.head())
print(df.info())
print(df.describe(include='all'))



  Make       Model  Year             Engine Fuel Type  Engine HP  \
0  BMW  1 Series M  2011  premium unleaded (required)      335.0   
1  BMW    1 Series  2011  premium unleaded (required)      300.0   
2  BMW    1 Series  2011  premium unleaded (required)      300.0   
3  BMW    1 Series  2011  premium unleaded (required)      230.0   
4  BMW    1 Series  2011  premium unleaded (required)      230.0   

   Engine Cylinders Transmission Type     Driven_Wheels  Number of Doors  \
0               6.0            MANUAL  rear wheel drive              2.0   
1               6.0            MANUAL  rear wheel drive              2.0   
2               6.0            MANUAL  rear wheel drive              2.0   
3               6.0            MANUAL  rear wheel drive              2.0   
4               6.0            MANUAL  rear wheel drive              2.0   

                         Market Category Vehicle Size Vehicle Style  \
0  Factory Tuner,Luxury,High-Performance      Compact         C

In [12]:

                 # Identifier types de variables
numeriques = df.select_dtypes(include=['int64', 'float64']).columns
catégorielles = df.select_dtypes(include=['object']).columns
print("Variables numériques :", list(numeriques))
print("Variables catégorielles :", list(catégorielles))



Variables numériques : ['Year', 'Engine HP', 'Engine Cylinders', 'Number of Doors', 'highway MPG', 'city mpg', 'Popularity', 'MSRP']
Variables catégorielles : ['Make', 'Model', 'Engine Fuel Type', 'Transmission Type', 'Driven_Wheels', 'Market Category', 'Vehicle Size', 'Vehicle Style']


In [15]:
                #identification  Valeurs manquantes
print(df.isnull().sum().sort_values(ascending=False))

Market Category      3742
Engine HP              69
Engine Cylinders       30
Number of Doors         6
Engine Fuel Type        3
Model                   0
Year                    0
Make                    0
Driven_Wheels           0
Transmission Type       0
Vehicle Size            0
Vehicle Style           0
highway MPG             0
city mpg                0
Popularity              0
MSRP                    0
dtype: int64


In [17]:
                #  identificationDoublons
print("Nombre de doublons :", df.duplicated().sum())

Nombre de doublons : 715


In [31]:
# 1. Recherche de doublons
duplicates = df[df.duplicated()]


In [32]:
# 2. Valeurs aberrantes ou incohérentes
#    -> Analyse des distributions pour détecter anomalies
summary_stats = df.describe()


In [33]:
# Rechercher des incohérences spécifiques
# Engine HP < 50 ou > 1000 peut être suspect
hp_outliers = df[(df["Engine HP"] < 50) | (df["Engine HP"] > 1000)]


In [34]:

# Number of Doors : vérifier que ce sont bien des valeurs entières et réalistes (ex: 2, 4)
door_outliers = df[~df["Number of Doors"].isin([2.0, 3.0, 4.0, 5.0])]

In [27]:

# Engine Cylinders : valeurs possibles généralement entre 3 et 12
cylinder_outliers = df[(df["Engine Cylinders"] < 3) | (df["Engine Cylinders"] > 12)]

In [35]:

# 3. Corrélations entre les variables numériques
correlations = df.corr(numeric_only=True)

In [30]:

# Affichage des résultats
{
    "nb_duplicated_rows": len(duplicates),
    "hp_outliers_count": len(hp_outliers),
    "door_outliers_count": len(door_outliers),
    "cylinder_outliers_count": len(cylinder_outliers),
    "correlation_matrix": correlations[["MSRP"]].sort_values(by="MSRP", ascending=False)
}


{'nb_duplicated_rows': 715,
 'hp_outliers_count': 3,
 'door_outliers_count': 6,
 'cylinder_outliers_count': 59,
 'correlation_matrix':                       MSRP
 MSRP              1.000000
 Engine HP         0.662008
 Engine Cylinders  0.531312
 Year              0.227590
 Popularity       -0.048476
 Number of Doors  -0.126635
 city mpg         -0.157676
 highway MPG      -0.160043}

oici ce que j'ai trouvé après l'analyse du fichier :
 1. Doublons

    715 lignes sont entièrement dupliquées dans le dataset.

 2. Valeurs aberrantes / incohérentes

    Engine HP : 3 valeurs sont suspectes (hors de l’intervalle [50, 1000]).

    Number of Doors : 6 lignes ont un nombre de portes non standard (autre que 2, 3, 4 ou 5).

    Engine Cylinders : 59 valeurs sont en dehors de la plage typique [3, 12].

3. Corrélations avec MSRP (prix du véhicule)

Les variables les plus corrélées avec le prix (MSRP) :

    Engine HP : +0.66

    Engine Cylinders : +0.53

    Year : +0.23

Corrélations négatives :

    highway MPG, city mpg : environ -0.16

    Popularity : très faible et légèrement négative -0.05

 4.  Plusieurs colonnes présentent des valeurs manquantes :

    Engine Fuel Type : 3 valeurs manquantes

    Engine HP : 69 valeurs manquantes

    Engine Cylinders : 30 valeurs manquantes

    Number of Doors : 6 valeurs manquantes

    Market Category : 3742 valeurs manquantes (beaucoup)

 5. colonnes utiles


    Engine HP, Engine Cylinders, Year, Transmission Type, Driven_Wheels, Vehicle Size, Vehicle Style

In [37]:
     #NOUS ALLONS PROCEDE A UN NETTOYAGE DE DONN2E


# Copie du dataframe original pour le nettoyage
df_cleaned = df.copy()


In [38]:

             # Suptression des DOUBLONS
df_cleaned = df_cleaned.drop_duplicates()

             #Suptression des valeurs aberrantes

df_cleaned = df_cleaned[(df_cleaned["Engine HP"] >= 50) & (df_cleaned["Engine HP"] <= 1000)]


df_cleaned = df_cleaned[(df_cleaned["Engine Cylinders"] >= 3) & (df_cleaned["Engine Cylinders"] <= 12)]


df_cleaned = df_cleaned[df_cleaned["Number of Doors"].isin([2.0, 3.0, 4.0, 5.0])]


df_cleaned["Engine HP"].fillna(df_cleaned["Engine HP"].mean(), inplace=True)


df_cleaned["Engine Cylinders"].fillna(df_cleaned["Engine Cylinders"].median(), inplace=True)

            #NOMBRE DE PORTE
df_cleaned["Number of Doors"].fillna(df_cleaned["Number of Doors"].mode()[0], inplace=True)
df_cleaned["Engine Fuel Type"].fillna("unknown", inplace=True)
df_cleaned["Market Category"].fillna("unknown", inplace=True)

df_cleaned.shape


C:\Users\kapna\AppData\Local\Temp\ipykernel_5320\56012322.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned["Engine HP"].fillna(df_cleaned["Engine HP"].mean(), inplace=True)
C:\Users\kapna\AppData\Local\Temp\ipykernel_5320\56012322.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh

(11084, 16)

 LE NETOYAGE EST TERMINE 
 
 RESUME :

    Doublons supprimés : 715 lignes

    Valeurs aberrantes supprimées : HP, Cylinders, Doors filtrés

    Valeurs manquantes imputées/remplacées :

        Engine HP → moyenne

        Engine Cylinders → médiane

        Number of Doors → mode

        Engine Fuel Type, Market Category → "unknown"

Taille finale du dataset nettoyé : 11 084 lignes, 16 colonnes

## 2. Nettoyage avancé
- Homogénéiser le format des prix
- Normaliser la cylindrée et la puissance
- Corriger les formats de date
- Uniformiser marques et modèles
- Questions : unités de puissance, traitement des lignes incohérentes


Exemples de nettoyage :
Prix : convertir "20,000 USD", "23k", "€19.5K" → nombre homogène

Cylindrée : "2.0L", "2000cc", "120 hp" → puissance normalisée

Dates : corriger les années absurdes (ex: fabrication après la vente)

Marques et modèles : uniformiser les noms ("VW", "Volkswagen", "volks wagen")

In [42]:
# Exemple de nettoyage des prix
#df['price_clean'] = df['price'].apply(clean_price)
price = pd.Series([
    "20,000 USD", "23k", "€19.5K", "$15,300", "18.2k", "22000", "€ 24.000", "25 K USD", "£21k"
])
print(price)

0    20,000 USD
1           23k
2        €19.5K
3       $15,300
4         18.2k
5         22000
6      € 24.000
7      25 K USD
8          £21k
dtype: object


In [44]:
# nettoyage de prix
import re

def clean_price(value):
    if pd.isnull(value):
        return None
    # Enlever les symboles monétaires et lettres inutiles
   # value = str(value).lower().replace(',', '').replace('€', '').replace('$', '').replace('usd', '').replace('£', '').strip()
    value = str(value).lower()
    value = value.replace('€', '').replace('$', '').replace('usd', '').replace('£', '').strip()
    value = value.replace('.', '') if ',' not in value else value  # pour "24.000"
    value = value.replace(',', '').replace(' ', '')
    
    # Si 'k' présent, convertir en milliers
    match = re.match(r'(\d+(\.\d+)?)[ ]*k', value)
    if match:
        return round(float(match.group(1)) * 1000)
    
    # Si nombre simple
    try:
        return round(float(value))
    except ValueError:
        return None

# Appliquer au jeu d'exemple
cleaned_price = price.apply(clean_price)
cleaned_price


0     20000
1     23000
2    195000
3     15300
4    182000
5     22000
6     24000
7     25000
8     21000
dtype: int64

## 3. Feature Engineering (création de variables)
- Calcul de l'âge du véhicule
- Catégorisation du segment (compact, SUV, etc.)
- Encodage des variables catégorielles
- Questions : relation âge/prix, influence du segment


Exemples :
Age du véhicule = année actuelle - année de fabrication

Segment du véhicule (ex: compact, SUV, utilitaire) à partir du modèle

Encodeur ordinal ou one-hot pour les catégories (carburant, boîte auto/manuelle, etc.)

In [ ]:
# Création de nouvelles variables
# df['age'] = current_year - df['year']

## 4. Analyse exploratoire (EDA)
- Corrélations entre variables numériques
- Distribution du prix par segment et type de carburant
- Identification des outliers sur le prix et le kilométrage
- Analyse de la distribution des variables (histogrammes, boxplots)
- Analyse des interactions entre variables (scatter matrix)

**Questions à se poser** :
- Quelles variables numériques sont les plus corrélées avec le prix ?
- Le prix suit-il une relation linéaire ou log-linéaire avec l'âge, le kilométrage et la puissance ?
- Y a-t-il des effets de seuil ou de segments (par exemple SUV vs citadine) sur la distribution des prix ?
- Comment le kilométrage impacte-t-il le prix selon l'âge du véhicule ?
- Y a-t-il des relations non linéaires entre la puissance moteur et le prix ?
- La localisation géographique ou la date d'inscription influence-t-elle significativement le prix ?
- Quels attributs catégoriels (carburant, transmission, couleur, nombre de portes) ont un impact sur le prix ?
- Y a-t-il des valeurs aberrantes dans d'autres variables (kilométrage, âge) qui pourraient fausser les analyses ?
- Les variables formant des paires montrent-elles des interactions (par exemple âge vs puissance) ?
- Faut-il transformer certaines variables (log, racine carrée) pour linéariser leur relation avec le prix ?

In [ ]:
# Visualisations
# import matplotlib.pyplot as plt

## 5. Modélisation (sans scikit-learn)
- Régression linéaire avec NumPy / statsmodels
- Évaluation performance sur validation
- Questions : variables importantes, sur-/sous-apprentissage

In [ ]:
# Modèle de régression
# import statsmodels.api as sm

## 6. Analyse des erreurs et des résidus
- Distribution des erreurs par segment
- Résidus vs prix réel
- Détection d'outliers non captés


In [ ]:
# Analyse des résidus
# residuals = model.predict(X) - y_true

## 7. Synthèse des résultats
- Importance des variables
- Exemples de bonnes/mauvaises prédictions
- Limites et pistes d'amélioration
